# Aula04.Ex02- Exemplo - Esfera - Transformação Geométrica 3D

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [66]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [67]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Nós estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Iremos aprender GLSL conforme a necessidade do curso. Usarmos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec2.
* Definindo uma variável chamada mat_transformation do tipo mat4 (matriz 4x4). Use ela como matriz de transformação, resultante de uma sequências de outras transformações (e.g. rotação + translação)
* Usamos vec2, pois nosso programa (na CPU) irá enviar apenas duas coordenadas para plotar um ponto. Podemos mandar três coordenadas (vec3) e até mesmo quatro coordenadas (vec4).
* void main() é o ponto de entrada do nosso programa (função principal)
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso nós combinamos nossa variável vec2 com uma variável vec4. Além disso, nós modificamos nosso vetor com base em uma matriz de transformação.

In [68]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        varying vec2 out_texture;
                
        uniform mat4 mat_transform;        
        
        void main(){
            gl_Position = mat_transform * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal)
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, nós vamos criar uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [69]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [70]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [71]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [72]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [73]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [74]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [75]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


### Modelando (Esfera ou .obj)

Função para carregar o .obj

In [76]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

Função para carregar a textura

In [77]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

In [78]:
is_sphere = False

obj_path = "./monstro/monstro.obj"
texture_path = "./monstro/monstro.jpg"

vertices_list = []    
textures_coord_list = []

In [79]:

load_model_from_file(obj_path)

{'vertices': [['0.901859', '3.548173', '-0.090302'],
  ['0.136334', '3.548173', '-0.090302'],
  ['0.892266', '3.465984', '0.274386'],
  ['0.145928', '3.465984', '0.274386'],
  ['0.519097', '3.461450', '0.579318'],
  ['0.519097', '3.461450', '-0.181091'],
  ['0.880310', '3.889165', '-0.083747'],
  ['0.157883', '3.889165', '-0.083747'],
  ['0.930070', '3.801512', '0.233011'],
  ['0.108123', '3.801512', '0.233011'],
  ['0.519097', '3.924544', '0.486176'],
  ['0.519097', '3.916883', '-0.115515'],
  ['0.827093', '3.262751', '0.406339'],
  ['0.211101', '3.262751', '0.406339'],
  ['0.519097', '3.262751', '0.567605'],
  ['0.957703', '3.336650', '-0.077062'],
  ['0.080490', '3.336650', '-0.077062'],
  ['0.519097', '3.262751', '-0.211567'],
  ['0.591551', '2.810722', '-0.054034'],
  ['0.446643', '2.810722', '-0.054034'],
  ['0.519097', '2.810722', '-0.076825'],
  ['0.567692', '2.810722', '0.252840'],
  ['0.470501', '2.810722', '0.252840'],
  ['0.519097', '2.810722', '0.291498'],
  ['1.037865', '

In [80]:
if is_sphere:
    PI = 3.141592
    r = 0.5 # raio
    num_sectors = 30 # qtd de sectors (longitude)
    num_stacks = 30 # qtd de stacks (latitude)

    # grid sectos vs stacks (longitude vs latitude)
    sector_step=(PI*2)/num_sectors # variar de 0 até 2π
    stack_step=(PI)/num_stacks # variar de 0 até π

    # Entrada: angulo de longitude, latitude, raio
    # Saida: coordenadas na esfera
    def F(u,v,r):
        x = r*math.sin(v)*math.cos(u)
        y = r*math.sin(v)*math.sin(u)
        z = r*math.cos(v)
        return (x,y,z)

    # vamos gerar um conjunto de vertices representantes poligonos
    # para a superficie da esfera.
    # cada poligono eh representado por dois triangulos
    for i in range(0,num_sectors): # para cada sector (longitude)
        
        for j in range(0,num_stacks): # para cada stack (latitude)
            
            
            
            u = i * sector_step # angulo setor
            v = j * stack_step # angulo stack
            
            un = 0 # angulo do proximo sector
            if i+1==num_sectors:
                un = PI*2
            else: un = (i+1)*sector_step
                
            vn = 0 # angulo do proximo stack
            if j+1==num_stacks:
                vn = PI
            else: vn = (j+1)*stack_step
            
            # verticies do poligono
            p0=F(u, v, r)
            p1=F(u, vn, r)
            p2=F(un, v, r)
            p3=F(un, vn, r)
            
            # triangulo 1 (primeira parte do poligono)
            vertices_list.append(p0)
            vertices_list.append(p2)
            vertices_list.append(p1)
            
            # triangulo 2 (segunda e ultima parte do poligono)
            vertices_list.append(p3)
            vertices_list.append(p1)
            vertices_list.append(p2)
else:
    modelo = load_model_from_file(obj_path)

    ### inserindo vertices do modelo no vetor de vertices
    print('Processando modelo caixa.obj. Vertice inicial:', len(vertices_list))
    for face in modelo['faces']:
        for vertice_id in face[0]:
            vertices_list.append( modelo['vertices'][vertice_id-1] )
        for texture_id in face[1]:
            textures_coord_list.append( modelo['texture'][texture_id-1] )
    print('Processando modelo caixa.obj. Vertice final:', len(vertices_list))

    ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
    load_texture_from_file(0, texture_path)

Processando modelo caixa.obj. Vertice inicial: 0
Processando modelo caixa.obj. Vertice final: 6108


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Nós agora vamos requisitar dois slots.
* Um para enviar coordenadas dos vértices.
* Outros para enviar coordenadas de texturas.

In [81]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)


###  Enviando coordenadas de vértices para a GPU


In [82]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)


###  Enviando coordenadas de textura para a GPU

In [83]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

In [84]:
def desenha_modelo(texture_id, init, count):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glDrawArrays(GL_TRIANGLES, init, count)

###  Vamos pegar a localização da variável color (uniform) do Fragment Shader para que possamos alterá-la em nosso laço da janela!

In [85]:
loc_color = glGetUniformLocation(program, "color")

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.


Usaremos o GL_TRIANGLE para modelar os triângulos (que formarão outros polígonos) da superfície da nossa esfera.

In [86]:
glEnable(GL_DEPTH_TEST) ### importante para 3D

In [87]:
def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

def get_rotation_x(angle):
    return np.array([1.0, 0.0, 0.0, 0.0,
                     0.0, math.cos(angle), -math.sin(angle), 0.0,
                     0.0, math.sin(angle), math.cos(angle), 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)

def get_rotation_y(angle):
    return np.array([math.cos(angle), 0.0, math.sin(angle), 0.0,
                     0.0, 1.0, 0.0, 0.0,
                     -math.sin(angle), 0.0, math.cos(angle), 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)

def get_rotation_z(angle):
    return np.array([math.cos(angle), -math.sin(angle), 0.0, 0.0,
                     math.sin(angle), math.cos(angle), 0.0, 0.0,
                     0.0, 0.0, 1.0, 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)

def get_translation(x,y,z):
    return np.array([1.0, 0.0, 0.0, x,
                     0.0, 1.0, 0.0, y,
                     0.0, 0.0, 1.0, z,
                     0.0, 0.0, 0.0, 1.0], np.float32)

def get_scale(x,y,z):
    return np.array([x, 0.0, 0.0, 0.0,
                     0.0, y, 0.0, 0.0,
                     0.0, 0.0, z, 0.0,
                     0.0, 0.0, 0.0, 1.0], np.float32)

In [88]:
mat_transform = multiplica_matriz(get_scale(0.5, 0.5, 0.5),np.identity(4, np.float32))

In [89]:
def key_event(window, key, scancode, action, mods):
    global mat_transform
    print(key)

    if action:
        if key == 256:
            glfw.set_window_should_close(window,True)
        if key == 90:
            mat_transform = multiplica_matriz(get_scale(1.1, 1.1, 1.1), mat_transform)
        if key == 88:
            mat_transform = multiplica_matriz(get_scale(0.9, 0.9, 0.9), mat_transform)
        if key == 65:
            mat_transform = multiplica_matriz(get_rotation_y(0.1), mat_transform)
        if key == 68:
            mat_transform = multiplica_matriz(get_rotation_y(-0.1), mat_transform)
        if key == 87:
            mat_transform = multiplica_matriz(get_rotation_x(0.1), mat_transform)
        if key == 83:
            mat_transform = multiplica_matriz(get_rotation_x(-0.1), mat_transform)
        if key == 67:
            mat_transform = multiplica_matriz(get_rotation_z(0.1), mat_transform)
        if key == 86:
            mat_transform = multiplica_matriz(get_rotation_z(-0.1), mat_transform)
        if key == 263:
            mat_transform = multiplica_matriz(get_translation(-0.1, 0.0, 0.0), mat_transform)
        if key == 262:
            mat_transform = multiplica_matriz(get_translation(0.1, 0.0, 0.0), mat_transform)
        if key == 265:
            mat_transform = multiplica_matriz(get_translation(0.0, 0.1, 0.0), mat_transform)
        if key == 264:
            mat_transform = multiplica_matriz(get_translation(0.0, -0.1, 0.0), mat_transform)

glfw.set_key_callback(window,key_event)


### Nesse momento, nós exibimos a janela!

In [90]:
glfw.show_window(window)

while not glfw.window_should_close(window):

    glfw.poll_events() 
        
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)

    #glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
   
    # mat_transform = multiplica_matriz(get_scale(0.5, 0.5, 0.5), mat_transform)

    loc = glGetUniformLocation(program, "mat_transform")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)

    desenha_modelo(0, 0, len(vertices_list))
    
    # for triangle in range(0,len(vertices),3):
       
    #     random.seed( triangle )
    #     R = random.random()
    #     G = random.random()
    #     B = random.random()  
    #     glUniform4f(loc_color, R, G, B, 1.0)
        
    #     glDrawArrays(GL_TRIANGLES, triangle, 3)     


    
    glfw.swap_buffers(window)

glfw.terminate()

343
340
340
343


KeyboardInterrupt: 

# Exercício

Modifique esse código para aplicar outras transformações (de sua escolha) conforme eventos do teclado.